__Video resources__

* https://www.youtube.com/watch?v=ZBD9he4Zp1E
* https://www.youtube.com/watch?v=Npv180dQ_4Y
* https://www.youtube.com/watch?v=NqYY0PJbD3s



__Read this__
* http://csbio.unc.edu/mcmillan/Comp555S16/Lecture14.html
* https://www.cs.cmu.edu/~ckingsf/bioinfo-lectures/gaps.pdf
* http://www.bioinfo.rpi.edu/bystrc/courses/biol4540/lecture4.pdf

In [1]:
import pandas as pd
import numpy as np
def createScorMat(match = 1,mismatch = 0,nts = ['A','C','T','G']):
    """
    Create scoring matrix for nucleotides with a given match and mismatch values
    """
    scoringMatrix = np.zeros((len(nts),len(nts)),)
    np.fill_diagonal(scoringMatrix,match)
    scoringMatrix
    scoringMatrix[scoringMatrix == 0] = mismatch
    scoringMatrix = pd.DataFrame(scoringMatrix,index=nts,columns=nts)
    return scoringMatrix

def gapMatrix(x,y,initY=-11,initX=-11):
    m = len(x) + 1
    n = len(y) + 1
    scoringMat=np.zeros((n,m)) # Main scoring matrix
    #scoringMat[:][0] = [0] + [-i + initX for i in range(1,m)]
    scoringMat[0][1:] = [initX] + [-i + initX for i in range(1,m-1)]
    #scoringMat[:,0] = [0] + [-i + initY for i in range(1,n)]
    scoringMat[1:,0] = [-i + initY for i in range(0,n-1)]
    #scoringMat[0,0]=start
    return scoringMat


def affineBacktrack(x,y,Mm,My,Mx,debug=False):
    i = len(x)
    j = len(y)
    w=''
    z=''
    scoreMax = max([Mm[j,i],Mx[j,i],My[j,i]])
    while i*j > 0:
        if debug: print(i,j,w[::-1],z[::-1])
        if x[i-1] == y[j-1]:
            w += x[i-1]
            z += y[j-1]
            i -= 1
            j -= 1
        else:
            whichMax = np.argmax([Mm[j,i],Mx[j,i],My[j,i]])
            if whichMax == 0:
                w += x[i-1]
                z += y[j-1]
                i -= 1
                j -= 1
            elif  whichMax == 1:
                w += x[i-1]
                z += '-'
                i -= 1
            else:
                w += '-'
                z += y[j-1]
                j -= 1
    if j==0 and i>0:
        w = x[:i]+w[::-1]
        z = '-' * i + z[::-1]
    elif i==0 and j>0:
        z = y[:i]+z[::-1]
        w = '-' * j + w[::-1]
    else:
        w = w[::-1]
        z = z[::-1]
    return w,z,scoreMax


def vizScoringMat(x,y,scoringMat):
    return pd.DataFrame(scoringMat,index=['']+[_ for _ in y],
             columns=['']+[_ for _ in x])

def readBlosum62():
    '''
    Read scoring matrix
    '''
    with open('blosum62.txt') as matrix_file:
        matrix = matrix_file.read()
        lines = matrix.strip().split('\n')

    blosum={}
    cols = lines[0].split()

    for row in lines[1:]:       
        idx = row[0]
        vals = row[1:].split()
        vals = [int(_) for _ in vals]
        #print(idx,vals)
        blosum[idx]=vals

    blosum = pd.DataFrame.from_dict(blosum,orient='index',columns=cols)
    return blosum

In [10]:
def gapScoringMatrices(x,y,subMat,gapOpen,gapExt):
    """
    Working
    """
    #### Initiate matrices
    Mmatrix = gapMatrix(x,y,gapOpen,gapOpen)
    Ix = gapMatrix(x,y,gapOpen,-np.inf)
    Iy = gapMatrix(x,y,-np.inf,gapOpen)

    m= len(x)
    n= len(y)

    # print(vizScoringMat(x,y,Ix))
    # print(vizScoringMat(x,y,Mmatrix))
    # print(vizScoringMat(x,y,Iy))

    #### Fill matrices
    for j in range(1,n+1):
        for i in range(1,m+1):


            #### Ix
            X = [Mmatrix[j-1,i] + gapOpen,
                 Ix[j-1,i] + gapExt]
            Ix[j,i] = max(X)
            #### Iy
            Y = [Mmatrix[j,i-1] + gapOpen,
                 Iy[j,i-1] + gapExt]
            Iy[j,i] = max(Y)
            ###
                    #### M matrix
            M = [Mmatrix[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]],
                     Ix[j,i], #+ subMat.loc[x[i-1]][y[j-1]],
                     Iy[j,i]]# + subMat.loc[x[i-1]][y[j-1]]]
            Mmatrix[j,i] = max(M)


#     print(vizScoringMat(x,y,Ix))
#     print(vizScoringMat(x,y,Mmatrix))
#     print(vizScoringMat(x,y,Iy))
    return Mmatrix, Ix, Iy

def gapAlign(x,y,subMat,gapOpen,gapExt):
    Mid,Upper,Left = gapScoringMatrices(x,y,subMat,gapOpen,gapExt)
    wAl, vAl, maxScore = affineBacktrack(x,y,Mid,Upper,Left)
    return wAl, vAl, maxScore

In [12]:
#### Start here
x='PRTEINS'
y='PRTWPSEIN'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
blosum = readBlosum62()
subMat = blosum
#### affine Gap Align
mid, up, low = gapScoringMatrices(x,y,subMat,gapOpen,gapExt)

print(vizScoringMat(x,y,up))
print(vizScoringMat(x,y,mid))
print(vizScoringMat(x,y,low))


            P     R     T     E     I     N     S
    0.0  -inf  -inf  -inf  -inf  -inf  -inf  -inf
P -11.0 -22.0 -23.0 -24.0 -25.0 -26.0 -27.0 -28.0
R -12.0  -4.0 -15.0 -16.0 -17.0 -18.0 -19.0 -20.0
T -13.0  -5.0   1.0 -10.0 -11.0 -12.0 -13.0 -14.0
W -14.0  -6.0   0.0   6.0  -5.0  -6.0  -7.0  -8.0
P -15.0  -7.0  -1.0   5.0   3.0  -7.0  -8.0  -9.0
S -16.0  -8.0  -2.0   4.0   2.0   0.0  -9.0 -10.0
E -17.0  -9.0  -3.0   3.0   1.0  -1.0   1.0  -6.0
I -18.0 -10.0  -4.0   2.0   0.0  -2.0   0.0   1.0
N -19.0 -11.0  -5.0   1.0  -1.0   2.0  -1.0   0.0
            P     R     T     E     I     N     S
    0.0 -11.0 -12.0 -13.0 -14.0 -15.0 -16.0 -17.0
P -11.0   7.0  -4.0  -5.0  -6.0  -7.0  -8.0  -9.0
R -12.0  -4.0  12.0   1.0   0.0  -1.0  -2.0  -3.0
T -13.0  -5.0   1.0  17.0   6.0   5.0   4.0   3.0
W -14.0  -6.0   0.0   6.0  14.0   3.0   2.0   1.0
P -15.0  -7.0  -1.0   5.0   5.0  11.0   1.0   1.0
S -16.0  -8.0  -2.0   4.0   5.0   3.0  12.0   5.0
E -17.0  -9.0  -3.0   3.0   9.0   2.0   3.0  12.0


In [13]:
#### Start here
x='YHFDVPDCWAHRYWVENPQAIAQMEQICFNWFPSMMMKQPHVFKVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE'
y='YHEDVAHEDAIAQMVNTFGFVWQICLNQFPSMMMKIYWIAVLSAHVADRKTWSKHMSCRWLPIISATCARMRVRTVWE'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
blosum = readBlosum62()
subMat = blosum

#### affine Gap Align

mid, up, low = gapScoringMatrices(x,y,subMat,gapOpen,gapExt)

print(vizScoringMat(x,y,up))
print(vizScoringMat(x,y,mid))
print(vizScoringMat(x,y,low))


             Y     H     F     D     V     P     D     C     W  ...     T  \
     0.0  -inf  -inf  -inf  -inf  -inf  -inf  -inf  -inf  -inf  ...  -inf   
Y  -11.0 -22.0 -23.0 -24.0 -25.0 -26.0 -27.0 -28.0 -29.0 -30.0  ... -87.0   
H  -12.0  -4.0 -15.0 -16.0 -17.0 -18.0 -19.0 -20.0 -21.0 -22.0  ... -79.0   
E  -13.0  -5.0   4.0  -7.0  -8.0  -9.0 -10.0 -11.0 -12.0 -13.0  ... -70.0   
D  -14.0  -6.0   3.0   1.0  -5.0 -10.0 -10.0  -8.0 -13.0 -14.0  ... -70.0   
..   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
R  -84.0 -76.0 -67.0 -69.0 -62.0 -57.0 -57.0 -57.0 -56.0 -54.0  ...  86.0   
T  -85.0 -77.0 -68.0 -70.0 -63.0 -58.0 -58.0 -58.0 -57.0 -55.0  ...  85.0   
V  -86.0 -78.0 -69.0 -71.0 -64.0 -59.0 -59.0 -59.0 -58.0 -56.0  ...  84.0   
W  -87.0 -79.0 -70.0 -72.0 -65.0 -60.0 -60.0 -60.0 -59.0 -57.0  ...  83.0   
E  -88.0 -80.0 -71.0 -73.0 -66.0 -61.0 -61.0 -61.0 -60.0 -58.0  ...  82.0   

       R     M     R      V      R      T      V      W      E  
    -inf  

In [17]:
#### Start here
x='PRTEINS'
y='PRTWPSEIN'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
blosum = readBlosum62()
subMat = blosum
#### affine Gap Align
mid, up, low = gapScoringMatrices(x,y,subMat,gapOpen,gapExt)

print(vizScoringMat(x,y,up))
print(vizScoringMat(x,y,mid))
print(vizScoringMat(x,y,low))


            P     R     T     E     I     N     S
    0.0  -inf  -inf  -inf  -inf  -inf  -inf  -inf
P -11.0 -22.0 -23.0 -24.0 -25.0 -26.0 -27.0 -28.0
R -12.0  -4.0 -15.0 -16.0 -17.0 -18.0 -19.0 -20.0
T -13.0  -5.0   1.0 -10.0 -11.0 -12.0 -13.0 -14.0
W -14.0  -6.0   0.0   6.0  -5.0  -6.0  -7.0  -8.0
P -15.0  -7.0  -1.0   5.0   3.0  -7.0  -8.0  -9.0
S -16.0  -8.0  -2.0   4.0   2.0   0.0  -9.0 -10.0
E -17.0  -9.0  -3.0   3.0   1.0  -1.0   1.0  -6.0
I -18.0 -10.0  -4.0   2.0   0.0  -2.0   0.0   1.0
N -19.0 -11.0  -5.0   1.0  -1.0   2.0  -1.0   0.0
            P     R     T     E     I     N     S
    0.0 -11.0 -12.0 -13.0 -14.0 -15.0 -16.0 -17.0
P -11.0   7.0  -4.0  -5.0  -6.0  -7.0  -8.0  -9.0
R -12.0  -4.0  12.0   1.0   0.0  -1.0  -2.0  -3.0
T -13.0  -5.0   1.0  17.0   6.0   5.0   4.0   3.0
W -14.0  -6.0   0.0   6.0  14.0   3.0   2.0   1.0
P -15.0  -7.0  -1.0   5.0   5.0  11.0   1.0   1.0
S -16.0  -8.0  -2.0   4.0   5.0   3.0  12.0   5.0
E -17.0  -9.0  -3.0   3.0   9.0   2.0   3.0  12.0


In [43]:
Mm = mid
Mx = low
My = up

In [44]:
i = len(x) 
j = len(y)
w=''
z=''
scoreMax = max([Mx[j,i],My[j,i],Mm[j,i]])
while i*j > 0:
    print(i,j,w,z[:)
    if x[i-1] == y[j-1]:
        w += x[i-1]
        z += y[j-1]
        i -= 1
        j -= 1
    else:
        whichMax = np.argmax([Mx[j,i],Mm[j,i],My[j,i]])
        if whichMax == 0:
            w += x[i-1]
            z += y[j-1]
            i -= 1
            j -= 1
        elif  whichMax == 1:
            w += x[i-1]
            z += '-'
            i -= 1
        else:
            w += '-'
            z += y[j-1]
            j -= 1
if j==0 and i>0:
    w = x[:i]+w[::-1]
    z = '-' * i + z[::-1]
elif i==0 and j>0:
    z = y[:i]+z[::-1]
    w = '-' * j + w[::-1]
else:
    w = w[::-1]
    z = z[::-1]
scoreMax,w,z

7 9  
6 8 S N
5 7 SN NI
4 7 SNI NI-
3 6 SNIE NI-E
2 6 SNIET NI-E-
1 6 SNIETR NI-E--


(8.0, '------PRTEINS', '---E-IN')

In [31]:
i,j

(0, 3)

In [5]:
x='PRTEINS'
y='PRTWPSEIN'
gapOpen = -11
gapExt = -1

#### Initiate matrices
Mmatrix = gapMatrix(x,y,gapOpen,gapOpen)
Ix = gapMatrix(x,y,gapOpen,-np.inf)
Iy = gapMatrix(x,y,-np.inf,gapOpen)

m= len(x)
n= len(y)

#     print(vizScoringMat(x,y,Ix))
#     print(vizScoringMat(x,y,Mmatrix))
#     print(vizScoringMat(x,y,Iy))

#### Fill matrices
for j in range(1,n+1):
    for i in range(1,m+1):
        #### M matrix
        M = [Mmatrix[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]],
                 Ix[j,i],# + subMat.loc[x[i-1]][y[j-1]],
                 Iy[j,i]]# + subMat.loc[x[i-1]][y[j-1]]]
        Mmatrix[j,i] = max(M)

        #### Ix
        X = [Mmatrix[j-1,i] + gapOpen,
             Ix[j-1,i] + gapExt]
        #### Iy
        Y = [Mmatrix[j,i-1] + gapOpen,
             Iy[j,i-1] + gapExt]
        ###
        Ix[j,i] = max(X)
        Iy[j,i] = max(Y)

In [64]:
Mmatrix,Ix,Iy

(array([[  0., -11., -12., -13., -14., -15., -16., -17.],
        [-11.,   7.,   0.,   0.,   0.,   0.,   0.,   0.],
        [-12.,   0.,  12.,   0.,   0.,   0.,   0.,   0.],
        [-13.,   0.,   0.,  17.,   0.,   0.,   0.,   1.],
        [-14.,   0.,   0.,   0.,  14.,   0.,   0.,   0.],
        [-15.,   0.,   0.,   0.,   0.,  11.,   0.,   0.],
        [-16.,   0.,   0.,   1.,   0.,   0.,  12.,   4.],
        [-17.,   0.,   0.,   0.,   6.,   0.,   0.,  12.],
        [-18.,   0.,   0.,   0.,   0.,  10.,   0.,   0.],
        [-19.,   0.,   0.,   0.,   0.,   0.,  16.,   1.]]),
 array([[  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-11., -22., -23., -24., -25., -26., -27., -28.],
        [-12.,  -4., -11., -11., -11., -11., -11., -11.],
        [-13.,  -5.,   1., -11., -11., -11., -11., -11.],
        [-14.,  -6.,   0.,   6., -11., -11., -11., -10.],
        [-15.,  -7.,  -1.,   5.,   3., -11., -11., -11.],
        [-16.,  -8.,  -2.,   4.,   2.,   0., -11., -11.],
        [-17

In [6]:
#### Start here
y='PRTEINS'
x='PRTWPSEIN'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
blosum = readBlosum62()
subMat = blosum

#### Initiate matrices
Mmatrix = gapMatrix(x,y,gapOpen,gapOpen)
Ix = gapMatrix(x,y,gapOpen,-np.inf)
Iy = gapMatrix(x,y,-np.inf,gapOpen)

m= len(x)
n= len(y)

print(vizScoringMat(x,y,Ix))
print(vizScoringMat(x,y,Mmatrix))
print(vizScoringMat(x,y,Iy))


           P    R    T    W    P    S    E    I    N
    0.0 -inf -inf -inf -inf -inf -inf -inf -inf -inf
P -11.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
R -12.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
T -13.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
E -14.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
I -15.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
N -16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
S -17.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
            P     R     T     W     P     S     E     I     N
    0.0 -11.0 -12.0 -13.0 -14.0 -15.0 -16.0 -17.0 -18.0 -19.0
P -11.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
R -12.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
T -13.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
E -14.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
I -15.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
N -16.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
S -17.0   0.0   0.0   0.0  

-1

In [8]:
for i in range(1, len(x)+1):
    for j in range(1, len(y)+1):
        lower_scores = [Ix[i-1,j] + gapExt, 
                        Mmatrix[i-1,j] + gapOpen]
        Ix[i,j] = max(lower_scores)


        upper_scores = [Iy[i,j-1] + gapExt,
                        Mmatrix[i,j-1] - gapOpen]
        Iy[i,j-1] = max(upper_scores)


        middle_scores = [Ix[i,j], 
                         Mmatrix[i-1,j-1] + blosum[x[i-1]][y[j-1]],
                         Iy[i,j]]
        S[1][i][j] = max(middle_scores)
        backtrack[1][i][j] = middle_scores.index(S[1][i][j])

NameError: name 'S' is not defined

In [76]:
###### Gap Matrices work well. Backtrack isn't working
#### Start here
y='PRTEINS'
x='PRTWPSEIN'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
blosum = readBlosum62()
subMat = blosum

#### Initiate matrices
Mmatrix = gapMatrix(x,y,gapOpen,gapOpen)
Ix = gapMatrix(x,y,gapOpen,-np.inf)
Iy = gapMatrix(x,y,-np.inf,gapOpen)

m= len(x)
n= len(y)

# print(vizScoringMat(x,y,Ix))
# print(vizScoringMat(x,y,Mmatrix))
# print(vizScoringMat(x,y,Iy))

#### Fill matrices
for j in range(1,n+1):
    for i in range(1,m+1):


        #### Ix
        X = [Mmatrix[j-1,i] + gapOpen,
             Ix[j-1,i] + gapExt]
        Ix[j,i] = max(X)
        #### Iy
        Y = [Mmatrix[j,i-1] + gapOpen,
             Iy[j,i-1] + gapExt]
        Iy[j,i] = max(Y)
        ###
                #### M matrix
        M = [Mmatrix[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]],
                 Ix[j,i], #+ subMat.loc[x[i-1]][y[j-1]],
                 Iy[j,i]]# + subMat.loc[x[i-1]][y[j-1]]]
        Mmatrix[j,i] = max(M)
        
        


print(vizScoringMat(x,y,Ix))
print(vizScoringMat(x,y,Mmatrix))
print(vizScoringMat(x,y,Iy))

            P     R     T     W     P     S     E     I     N
    0.0  -inf  -inf  -inf  -inf  -inf  -inf  -inf  -inf  -inf
P -11.0 -22.0 -23.0 -24.0 -25.0 -26.0 -27.0 -28.0 -29.0 -30.0
R -12.0  -4.0 -15.0 -16.0 -17.0 -18.0 -19.0 -20.0 -21.0 -22.0
T -13.0  -5.0   1.0 -10.0 -11.0 -12.0 -13.0 -14.0 -15.0 -16.0
E -14.0  -6.0   0.0   6.0  -5.0  -6.0  -7.0  -8.0  -9.0 -10.0
I -15.0  -7.0  -1.0   5.0   3.0  -6.0  -6.0  -2.0 -10.0  -9.0
N -16.0  -8.0  -2.0   4.0   2.0   0.0  -7.0  -3.0   2.0  -9.0
S -17.0  -9.0  -3.0   3.0   1.0  -1.0   1.0  -4.0   1.0   8.0
            P     R     T     W     P     S     E     I     N
    0.0 -11.0 -12.0 -13.0 -14.0 -15.0 -16.0 -17.0 -18.0 -19.0
P -11.0   7.0  -4.0  -5.0  -6.0  -7.0  -8.0  -9.0 -10.0 -11.0
R -12.0  -4.0  12.0   1.0   0.0  -1.0  -2.0  -3.0  -4.0  -5.0
T -13.0  -5.0   1.0  17.0   6.0   5.0   4.0   3.0   2.0   1.0
E -14.0  -6.0   0.0   6.0  14.0   5.0   5.0   9.0   0.0   2.0
I -15.0  -7.0  -1.0   5.0   3.0  11.0   3.0   2.0  13.0   2.0
N -16.0 

In [80]:
Mm = Mmatrix
Mx = Ix
My = Iy

i = len(x)
j = len(y)
w=''
z=''
scoreMax = max([Mm[j,i],Mx[j,i],My[j,i]])
while i*j > 0:
    print(i,j,w[::-1],z[::-1])
    if x[i-1] == y[j-1]:
        w += x[i-1]
        z += y[j-1]
        i -= 1
        j -= 1
    else:
        whichMax = np.argmax([Mm[j,i],Mx[j,i],My[j,i]])
        if whichMax == 0:
            w += x[i-1]
            z += y[j-1]
            i -= 1
            j -= 1
        elif  whichMax == 1:
            w += x[i-1]
            z += '-'
            i -= 1
        else:
            w += '-'
            z += y[j-1]
            j -= 1
if j==0 and i>0:
    w = x[:i]+w[::-1]
    z = '-' * i + z[::-1]
elif i==0 and j>0:
    z = y[:i]+z[::-1]
    w = '-' * j + w[::-1]
else:
    w = w[::-1]
    z = z[::-1]
    
print(i,j,w,z)

9 7  
8 6 N S
7 5 IN NS
6 4 EIN INS
5 3 SEIN EINS
4 2 PSEIN TEINS
3 1 WPSEIN RTEINS
2 0 PRTWPSEIN --PRTEINS


In [9]:
###### WOOOOOOORKS
#### Start here
x='YHFDVPDCWAHRYWVENPQAIAQMEQICFNWFPSMMMKQPHVFKVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE'
y='YHEDVAHEDAIAQMVNTFGFVWQICLNQFPSMMMKIYWIAVLSAHVADRKTWSKHMSCRWLPIISATCARMRVRTVWE'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
blosum = readBlosum62()
subMat = blosum

#### Initiate matrices
Mmatrix = gapMatrix(x,y,gapOpen,gapOpen)
Ix = gapMatrix(x,y,gapOpen,-np.inf)
Iy = gapMatrix(x,y,-np.inf,gapOpen)

m= len(x)
n= len(y)

# print(vizScoringMat(x,y,Ix))
# print(vizScoringMat(x,y,Mmatrix))
# print(vizScoringMat(x,y,Iy))

#### Fill matrices
for j in range(1,n+1):
    for i in range(1,m+1):


        #### Ix
        X = [Mmatrix[j-1,i] + gapOpen,
             Ix[j-1,i] + gapExt]
        Ix[j,i] = max(X)
        #### Iy
        Y = [Mmatrix[j,i-1] + gapOpen,
             Iy[j,i-1] + gapExt]
        Iy[j,i] = max(Y)
        ###
                #### M matrix
        M = [Mmatrix[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]],
                 Ix[j,i], #+ subMat.loc[x[i-1]][y[j-1]],
                 Iy[j,i]]# + subMat.loc[x[i-1]][y[j-1]]]
        Mmatrix[j,i] = max(M)

affineBacktrack(x,y,Mmatrix,Ix,Iy)

('---YHFDVPDCWAHRYWVENPQAIAQMEQICFNWFPSMMMKQPHVFKVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE',
 'DVAHEDAIAQMVNTFGFVWQICLNQFPSMMMKIYWIAVLSAHVADRKTWSKHMSCRWLPIISATCARMRVRTVWE',
 144.0)

In [74]:
###
affineBacktrack(x,y,Mmatrix,Ix,Iy)

('YHEDVAHEDAIAQMVNTFGFVWQICLNQFPSMMMKIYWIAVLSAHVADRKTWSKHMSCRWLPIISATCARMRVRTVWE',
 '---YHFDVPDCWAHRYWVENPQAIAQMEQICFNWFPSMMMKQPHVFKVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE',
 144.0)

-2

In [1]:
import pandas as pd
import numpy as np
def createScorMat(match = 1,mismatch = 0,nts = ['A','C','T','G']):
    """
    Create scoring matrix for nucleotides with a given match and mismatch values
    """
    scoringMatrix = np.zeros((len(nts),len(nts)),)
    np.fill_diagonal(scoringMatrix,match)
    scoringMatrix
    scoringMatrix[scoringMatrix == 0] = mismatch
    scoringMatrix = pd.DataFrame(scoringMatrix,index=nts,columns=nts)
    return scoringMatrix

#### Dynamic programming implementation
def scoringMatrix_local(x,y,subMat,indel):
    m = len(x) + 1
    n = len(y) + 1
    globMax = (0,0)
    scoringMat=np.zeros((n,m))
    for i in range(1,m):
        #print(x[i-1])
        for j in range(1,n):
            if x[i-1] == y[j-1]:
                #scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                scoringMat[j][i] = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            else:
                mismatch = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                left = scoringMat[j-1][i] + indel
                up = scoringMat[j][i-1] + indel
                scoringMat[j][i] = max(mismatch,left,up,0)
            if scoringMat[j][i] >= scoringMat[globMax[1]][globMax[0]]:
                globMax = (i,j)
    return scoringMat,globMax


def scoringMatrix_global(x,y,subMat,indel):
    m = len(x) + 1
    n = len(y) + 1
    scoringMat=np.zeros((n,m))
    scoringMat[:][0] = [0] + list(np.cumsum([indel for _ in x]))
    scoringMat[:,0] = [0] + list(np.cumsum([indel for _ in y]))
    for i in range(1,m):
        #print(x[i-1])
        for j in range(1,n):
            if x[i-1] == y[j-1]:
                #scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                scoringMat[j][i] = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            else:
                mismatch = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                left = scoringMat[j-1][i] + indel
                up = scoringMat[j][i-1] + indel
                scoringMat[j][i] = max(mismatch,left,up)
    return scoringMat


def backtrack_local(x,y,sm,maxLocs):
    i = maxLocs[0]
    j = maxLocs[1]
    w=''
    z=''
    z,w
    while i*j > 0:
        if x[i-1] == y[j-1]:
            w += x[i-1]
            z += y[j-1]
            i -= 1
            j -= 1
        else:
            left = sm[j-1][i]
            up = sm[j][i-1]
            diag = sm[j-1][i-1]
            whichmax = np.argmax([left,up,diag])
            if whichmax == 0:
                w += '-'
                z += y[j-1]
                j -= 1
            elif whichmax == 1:
                z += '-'
                w += x[i-1]
                i -= 1
            else:
                w += x[i-1]
                z += y[j-1]
                i -= 1
                j -= 1
        if sm[j][i] == 0:
            break
    return w[::-1],z[::-1]

def backtrack_global(x,y,sm,subMat,indel):
    i = len(x)
    j = len(y)
    w=''
    z=''
    z,w
    while i*j > 0:
        if x[i-1] == y[j-1]:
            w += x[i-1]
            z += y[j-1]
            i -= 1
            j -= 1
        else:
            left = sm[j-1][i] + indel
            up = sm[j][i-1] + indel
            diag = sm[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            whichmax = np.argmax([left,up,diag])
            if whichmax == 0:
                w += '-'
                z += y[j-1]
                j -= 1
            elif whichmax == 1:
                z += '-'
                w += x[i-1]
                i -= 1
            else:
                w += x[i-1]
                z += y[j-1]
                i -= 1
                j -= 1
    if j==0 and i>0:
        w = x[:i]+w[::-1]
        z = '-' * i + z[::-1]
    elif i==0 and j>0:
        z = y[:i]+z[::-1]
        w = '-' * j + w[::-1]
    else:
        w = w[::-1]
        z = z[::-1]
    return w,z

def locAl(x,y,subMat,indel):
    '''
    A wrapper for DP implementation of Smith-Waterman
    '''
    sm, maxLocs = scoringMatrix_local(x,y,subMat,indel)
    z,w = backtrack_local(x,y,sm,maxLocs)
    return z,w,sm[maxLocs[1],maxLocs[0]],sm

def globAl(x,y,subMat,indel):
    '''
    A wrapper for DP implementation of Smith-Waterman
    '''
    sm = scoringMatrix_global(x,y,subMat,indel)
    z,w = backtrack_global(x,y,sm,subMat,indel)
    return z,w,sm[len(y),len(x)]

def vizScoringMat(x,y,scoringMat):
    return pd.DataFrame(scoringMat,index=['']+[_ for _ in y],
             columns=['']+[_ for _ in x])

def readBlosum62():
    '''
    Read scoring matrix
    '''
    with open('blosum62.txt') as matrix_file:
        matrix = matrix_file.read()
        lines = matrix.strip().split('\n')

    blosum={}
    cols = lines[0].split()

    for row in lines[1:]:       
        idx = row[0]
        vals = row[1:].split()
        vals = [int(_) for _ in vals]
        #print(idx,vals)
        blosum[idx]=vals

    blosum = pd.DataFrame.from_dict(blosum,orient='index',columns=cols)
    return blosum

In [2]:
blosum = readBlosum62()

In [3]:
### Example from https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5
## Problem 3: Overlap Alignment
x='PRTEINS'
y='PRTWPSEIN'
indel = -8
subMat = blosum
locAl(x,y,subMat,indel)

('PRT',
 'PRT',
 17.0,
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., 12.,  4.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  4., 17.,  9.,  1.,  0.,  1.],
        [ 0.,  0.,  0.,  9., 14.,  6.,  0.,  0.],
        [ 0.,  7.,  0.,  1.,  8., 11.,  4.,  0.],
        [ 0.,  0.,  6.,  1.,  1.,  6., 12.,  8.],
        [ 0.,  0.,  0.,  5.,  6.,  0.,  6., 12.],
        [ 0.,  0.,  0.,  0.,  2., 10.,  2.,  4.],
        [ 0.,  0.,  0.,  0.,  0.,  2., 16.,  8.]]))

In [31]:
### Example from https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5
## Problem 3: Overlap Alignment
x='ATTGACCTGA'
y='ATCCTGA'
indel = -1
gap = -1
subMat = createScorMat(match = 1,mismatch = -2,nts = list(set(y) | set(x)))
globAl(x,y,subMat,indel)

('ATTGACCTGA', 'A-T--CCTGA', 4.0)

In [32]:
### Example from https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5
## Problem 3: Overlap Alignment
x='PRTEINS'
y='PRTWPSEIN'
indel = -11
gap = -1
subMat = blosum
sm,_ = scoringMatrix_local(x,y,subMat,indel)
backtrack_local(x,y,sm,(len(x),len(y)))

('PRTEINS', 'PS-EIN-')

In [33]:
#http://csbio.unc.edu/mcmillan/Comp555S16/Lecture14.html
#http://rosalind.info/problems/ba5j/

In [34]:
### Example from https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5
## Problem 3: Overlap Alignment
x='ACACT'
y='AAT'
indel = -1
gap = -3
subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
globAl(x,y,subMat,indel),

(('ACACT', 'A-A-T', 1.0),)

In [35]:
def initScoringTable(x,y,initY=indel,initX=indel,start=0):
    m = len(x) + 1
    n = len(y) + 1
    scoringMat=np.zeros((n,m)) # Main scoring matrix
    scoringMat[:][0] = [0] + list(np.cumsum([initX for _ in x]))
    scoringMat[:,0] = [0] + list(np.cumsum([initY for _ in y]))
    scoringMat[0,0]=start
    return scoringMat

In [36]:
gapExtension = -1
gapOpen = -3

### Follow this example 
https://www.youtube.com/watch?v=NqYY0PJbD3s

In [37]:
m= len(x)
n= len(y)

In [38]:
####
scoringMat=np.zeros((n,m)) # Main scoring matrix
inX = [gapOpen]
[inX.append(inX[i-1] + gapExtension) for i in range(1,m)]
scoringMat[:][0] = inX
scoringMat[:,0][1:] = [-np.inf  for i in range(1,n)]
scoringMat
scoringMat

array([[ -3.,  -4.,  -5.,  -6.,  -7.],
       [-inf,   0.,   0.,   0.,   0.],
       [-inf,   0.,   0.,   0.,   0.]])

In [39]:
####
scoringMat=np.zeros((n,m)) # Main scoring matrix
inY = [gapOpen]
[inY.append(inY[i-1] + gapExtension) for i in range(1,n)]
scoringMat[:,0] = inY
scoringMat[:][0][1:] = [-np.inf  for i in range(1,m)]
scoringMat

array([[ -3., -inf, -inf, -inf, -inf],
       [ -4.,   0.,   0.,   0.,   0.],
       [ -5.,   0.,   0.,   0.,   0.]])

In [40]:
inY = [gapOpen]
[inY.append(inY[i-1] + gapExtension) for i in range(1,m)]
inY

[-3, -4, -5, -6, -7]

In [41]:
def gapMatrix(x,y,initY=gap,initX=gap,start=0):
    m = len(x) + 1
    n = len(y) + 1
    scoringMat=np.zeros((n,m)) # Main scoring matrix
    scoringMat[:][0] = [0] + [-i + initX for i in range(1,m)]
    scoringMat[:,0] = [0] + [-i + initY for i in range(1,n)]
    scoringMat[0,0]=start
    return scoringMat

In [42]:
gapMatrix(x,y,-np.inf,gap,gap)

array([[ -3.,  -4.,  -5.,  -6.,  -7.,  -8.],
       [-inf,   0.,   0.,   0.,   0.,   0.],
       [-inf,   0.,   0.,   0.,   0.,   0.],
       [-inf,   0.,   0.,   0.,   0.,   0.]])

In [43]:
gapMatrix(x,y,gap,-np.inf,gap)

array([[ -3., -inf, -inf, -inf, -inf, -inf],
       [ -4.,   0.,   0.,   0.,   0.,   0.],
       [ -5.,   0.,   0.,   0.,   0.,   0.],
       [ -6.,   0.,   0.,   0.,   0.,   0.]])

#### Start here

In [44]:
x,y

('ACACT', 'AAT')

In [ ]:
epsilon = 1
sigma = 11

In [1]:
def gapMatrix(x,y,initY=gap,initX=gap,start=0):
    m = len(x) + 1
    n = len(y) + 1
    scoringMat=np.zeros((n,m)) # Main scoring matrix
    scoringMat[:][0] = [0] + [-i + initX for i in range(1,m)]
    scoringMat[:,0] = [0] + [-i + initY for i in range(1,n)]
    scoringMat[0,0]=start
    return scoringMat

def affineGapScoring(x,y,subMat,indel,gap):
    #### Initiate
    Mmatrix = gapMatrix(x,y,-np.inf,-np.inf,0)
    Ix = gapMatrix(x,y,gap,-np.inf,gap)
    Iy = gapMatrix(x,y,-np.inf,gap,gap)
    m= len(x)
    n= len(y)
    for j in range(1,n+1):
        for i in range(1,m+1):
            #### M matrix
            M = [Mmatrix[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]],
                     Ix[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]],
                     Iy[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]]]
            Mmatrix[j,i] = max(M)

            #### Ix
            X = [Mmatrix[j-1,i] + gap + indel,
                 Ix[j-1,i] + indel]
            #### Iy
            Y = [Mmatrix[j,i-1] + gap + indel,
                 Iy[j,i-1] + indel]
            ###
            Ix[j,i] = max(X)
            Iy[j,i] = max(Y)
    return Mmatrix, Ix, Iy

def affineBacktrack(x,y,Mm,Ix,Iy):
    i = len(x)
    j = len(y)
    w=''
    z=''
    scoreMax = max([Mm[j,i],Mx[j,i],My[j,i]])
    while i*j > 0:
        print(i,j,w[::-1],z[::-1])
        if x[i-1] == y[j-1]:
            w += x[i-1]
            z += y[j-1]
            i -= 1
            j -= 1
        else:
            whichMax = np.argmax([Mm[j,i],Mx[j,i],My[j,i]])
            if whichMax == 0:
                w += x[i-1]
                z += y[j-1]
                i -= 1
                j -= 1
            elif  whichMax == 1:
                w += x[i-1]
                z += '-'
                i -= 1
            else:
                w += '-'
                z += y[j-1]
                j -= 1
    if j==0 and i>0:
        w = x[:i]+w[::-1]
        z = '-' * i + z[::-1]
    elif i==0 and j>0:
        z = y[:i]+z[::-1]
        w = '-' * j + w[::-1]
    else:
        w = w[::-1]
        z = z[::-1]
    return w,z,scoreMax

NameError: name 'gap' is not defined

In [46]:
x='ACACT'
y='AAT'
gap = -3
indel = -1
subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))

Mm, Mx, My = affineGapScoring(x,y,subMat,indel,gap)
w,z, sm = affineBacktrack(x,y,Mm,Mx,My)

#### Result
print(vizScoringMat(x,y,Mm))
print(vizScoringMat(x,y,Mx))
print(vizScoringMat(x,y,My))
print(w,z, sm)

5 3  
4 2 T T
3 1 CT AT
          A    C    A    C    T
   0.0 -inf -inf -inf -inf -inf
A -inf  1.0 -5.0 -4.0 -7.0 -8.0
A -inf -3.0  0.0 -2.0 -5.0 -6.0
T -inf -6.0 -4.0 -1.0 -3.0 -4.0
          A    C    A     C     T
  -3.0 -inf -inf -inf  -inf  -inf
A -4.0 -inf -inf -inf  -inf  -inf
A -5.0 -3.0 -9.0 -8.0 -11.0 -12.0
T -6.0 -4.0 -4.0 -6.0  -9.0 -10.0
          A     C    A    C    T
  -3.0 -4.0  -5.0 -6.0 -7.0 -8.0
A -inf -inf  -3.0 -4.0 -5.0 -6.0
A -inf -inf  -7.0 -4.0 -5.0 -6.0
T -inf -inf -10.0 -8.0 -5.0 -6.0
ACACT --AAT -4.0


#### Rosalind example

https://www.ebi.ac.uk/Tools/services/web/toolresult.ebi?jobId=emboss_needle-I20201112-053202-0673-88247623-p1m

In [47]:
globAl(x,y,subMat,indel)

('ACACT', 'A-A-T', 1.0)

In [17]:
### Forget this
# def gapMatrix(x,y,initY=gapExt,initX=gapExt,start=0):
#     m = len(x) + 1
#     n = len(y) + 1
#     scoringMat=np.zeros((n,m)) # Main scoring matrix
#     scoringMat[:][0] = [0] + [-i + initX for i in range(1,m)]
#     scoringMat[:,0] = [0] + [-i + initY for i in range(1,n)]
#     scoringMat[0,0]=start
#     return scoringMat

In [68]:
def gapMatrix(x,y,initY=gapOpen,initX=gapOpen):
    m = len(x) + 1
    n = len(y) + 1
    scoringMat=np.zeros((n,m)) # Main scoring matrix
    #scoringMat[:][0] = [0] + [-i + initX for i in range(1,m)]
    scoringMat[0][1:] = [initX] + [-i + initX for i in range(1,m-1)]
    #scoringMat[:,0] = [0] + [-i + initY for i in range(1,n)]
    scoringMat[1:,0] = [-i + initY for i in range(0,n-1)]
    #scoringMat[0,0]=start
    return scoringMat

In [75]:
#### Start here
x='PRTEINS'
y='PRTWPSEIN'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
subMat = blosum

In [85]:
#### Start here
x='PRTEINS'
y='PRTWPSEIN'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
subMat = blosum

#### Initiate matrices
Mmatrix = gapMatrix(x,y,gapOpen,gapOpen)
Ix = gapMatrix(x,y,gapOpen,-np.inf)
Iy = gapMatrix(x,y,-np.inf,gapOpen)

m= len(x)
n= len(y)

print(vizScoringMat(x,y,Ix))
print(vizScoringMat(x,y,Mmatrix))
print(vizScoringMat(x,y,Iy))

           P    R    T    E    I    N    S
    0.0 -inf -inf -inf -inf -inf -inf -inf
P -11.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
R -12.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
T -13.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
W -14.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
P -15.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
S -16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
E -17.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
I -18.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
N -19.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
            P     R     T     E     I     N     S
    0.0 -11.0 -12.0 -13.0 -14.0 -15.0 -16.0 -17.0
P -11.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
R -12.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
T -13.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
W -14.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
P -15.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
S -16.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
E -17.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
I -18.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
N -19.0   0.0   0.0   0.0  

In [86]:
sigma = 11
epsilon = 1

In [90]:
#### Start here
x='PRTEINS'
y='PRTWPSEIN'
gapOpen = -11
gapExt = -1
#subMat = createScorMat(match = 1,mismatch = -1,nts = list(set(y) | set(x)))
subMat = blosum

#### Initiate matrices
Mmatrix = gapMatrix(x,y,gapOpen,gapOpen)
Ix = gapMatrix(x,y,gapOpen,-np.inf)
Iy = gapMatrix(x,y,-np.inf,gapOpen)

m= len(x)
n= len(y)

print(vizScoringMat(x,y,Ix))
print(vizScoringMat(x,y,Mmatrix))
print(vizScoringMat(x,y,Iy))

#### Fill matrices
for j in range(1,n+1):
    for i in range(1,m+1):
        #### M matrix
        M = [Mmatrix[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]],
                 Ix[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]],
                 Iy[j-1,i-1] + subMat.loc[x[i-1]][y[j-1]]]
        Mmatrix[j,i] = max(M)

        #### Ix
        X = [Mmatrix[j-1,i] + gapOpen,
             Ix[j-1,i] + gapExt]
        #### Iy
        Y = [Mmatrix[j,i-1] + gapOpen,
             Iy[j,i-1] + gapOpen]
        ###
        Ix[j,i] = max(X)
        Iy[j,i] = max(Y)


print(vizScoringMat(x,y,Ix))
print(vizScoringMat(x,y,Mmatrix))
print(vizScoringMat(x,y,Iy))

            P     R     T     E     I     N     S
    0.0  -inf  -inf  -inf  -inf  -inf  -inf  -inf
P -11.0 -22.0 -23.0 -24.0 -25.0 -26.0 -27.0 -28.0
R -12.0  -4.0 -24.0 -24.0 -25.0 -27.0 -28.0 -28.0
T -13.0  -5.0   1.0 -16.0 -24.0 -28.0 -28.0 -29.0
W -14.0  -6.0   0.0   6.0 -11.0 -22.0 -28.0 -27.0
P -15.0  -7.0  -1.0   5.0   3.0  -8.0 -20.0 -28.0
S -16.0  -8.0  -2.0   4.0   2.0   0.0 -10.0 -20.0
E -17.0  -9.0  -3.0   3.0   1.0  -1.0   1.0  -6.0
I -18.0 -10.0  -4.0   2.0   0.0  -2.0   0.0   1.0
N -19.0 -11.0  -5.0   1.0  -1.0   2.0  -1.0   0.0
            P     R     T     E     I     N     S
    0.0 -11.0 -12.0 -13.0 -14.0 -15.0 -16.0 -17.0
P -11.0   7.0 -13.0 -13.0 -14.0 -17.0 -17.0 -17.0
R -12.0 -13.0  12.0  -5.0 -13.0 -17.0 -17.0 -18.0
T -13.0 -13.0  -5.0  17.0   0.0 -11.0 -17.0 -16.0
W -14.0 -17.0  -8.0  -1.0  14.0   3.0  -9.0 -19.0
P -15.0  -7.0  -8.0  -1.0   5.0  11.0   1.0  -9.0
S -16.0 -16.0  -8.0   0.0   5.0   3.0  12.0   5.0
E -17.0 -17.0  -8.0  -3.0   9.0   2.0   3.0  12.0


In [ ]:
## From https://github.com/ZhangQiuxue/Rosalind/blob/master/096_GAFF.py

In [24]:
w=y
v=x

In [25]:
S = [[[0 for j in range(len(w)+1)] for i in range(len(v)+1)] for k in range(3)]
backtrack = [[[0 for j in range(len(w)+1)] for i in range(len(v)+1)] for k in range(3)]

In [26]:
epsilon = 1
sigma = 11
for i in range(1, len(v)+1):
        S[0][i][0] = -sigma - (i-1)*epsilon
        S[1][i][0] = -sigma - (i-1)*epsilon
        S[2][i][0] = -10*sigma
for j in range(1, len(w)+1):
    S[2][0][j] = -sigma - (j-1)*epsilon
    S[1][0][j] = -sigma - (j-1)*epsilon
    S[0][0][j] = -10*sigma
S

[[[0, -110, -110, -110, -110, -110, -110, -110, -110, -110],
  [-11, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-12, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-13, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-14, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-15, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-16, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-17, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, -11, -12, -13, -14, -15, -16, -17, -18, -19],
  [-11, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-12, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-13, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-14, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-15, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-16, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-17, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, -11, -12, -13, -14, -15, -16, -17, -18, -19],
  [-110, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-110, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-110, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-110, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-110, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-110, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [-110, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]

In [74]:
##https://github.com/ZhangQiuxue/Rosalind/blob/master/096_GAFF.py

In [22]:
for i in range(1, len(v)+1):
    for j in range(1, len(w)+1):
        lower_scores = [S[0][i-1][j] - epsilon, S[1][i-1][j] - sigma]
        S[0][i][j] = max(lower_scores)
        backtrack[0][i][j] = lower_scores.index(S[0][i][j])

        upper_scores = [S[2][i][j-1] - epsilon, S[1][i][j-1] - sigma]
        S[2][i][j] = max(upper_scores)
        backtrack[2][i][j] = upper_scores.index(S[2][i][j])

        middle_scores = [S[0][i][j], 
                         S[1][i-1][j-1] + blosum[v[i-1]][w[j-1]],
                         S[2][i][j]]
        S[1][i][j] = max(middle_scores)
        backtrack[1][i][j] = middle_scores.index(S[1][i][j])

In [23]:
S

[[[0, -110, -110, -110, -110, -110, -110, -110, -110, -110],
  [-11, -22, -23, -24, -25, -26, -27, -28, -29, -30],
  [-12, -4, -15, -16, -17, -18, -19, -20, -21, -22],
  [-13, -5, 1, -10, -11, -12, -13, -14, -15, -16],
  [-14, -6, 0, 6, -5, -6, -7, -8, -9, -10],
  [-15, -7, -1, 5, 3, -6, -6, -2, -10, -9],
  [-16, -8, -2, 4, 2, 0, -7, -3, 2, -9],
  [-17, -9, -3, 3, 1, -1, 1, -4, 1, 8]],
 [[0, -11, -12, -13, -14, -15, -16, -17, -18, -19],
  [-11, 7, -4, -5, -6, -7, -8, -9, -10, -11],
  [-12, -4, 12, 1, 0, -1, -2, -3, -4, -5],
  [-13, -5, 1, 17, 6, 5, 4, 3, 2, 1],
  [-14, -6, 0, 6, 14, 5, 5, 9, 0, 2],
  [-15, -7, -1, 5, 3, 11, 3, 2, 13, 2],
  [-16, -8, -2, 4, 2, 1, 12, 3, 2, 19],
  [-17, -9, -3, 3, 1, 1, 5, 12, 1, 8]],
 [[0, -11, -12, -13, -14, -15, -16, -17, -18, -19],
  [-110, -22, -4, -5, -6, -7, -8, -9, -10, -11],
  [-110, -23, -15, 1, 0, -1, -2, -3, -4, -5],
  [-110, -24, -16, -10, 6, 5, 4, 3, 2, 1],
  [-110, -25, -17, -11, -5, 3, 2, 1, 0, -1],
  [-110, -26, -18, -12, -6, -7, 0, -1, 

In [51]:
i,j = len(v), len(w)
v_aligned, w_aligned = v, w

# Get the maximum score, and the corresponding backtrack starting position.
matrix_scores = [S[0][i][j], S[1][i][j], S[2][i][j]]
max_score = max(matrix_scores)
backtrack_matrix = matrix_scores.index(max_score)

# Quick lambda function to insert indels.
insert_indel = lambda word, i: word[:i] + '-' + word[i:]


In [52]:
while i*j != 0:
    if backtrack_matrix == 0:  # Lower backtrack matrix conditions.
        if backtrack[0][i][j] == 1:
            backtrack_matrix = 1
        i -= 1
        w_aligned = insert_indel(w_aligned, j)

    elif backtrack_matrix == 1:  # Middle backtrack matrix conditions.
        if backtrack[1][i][j] == 0:
            backtrack_matrix = 0
        elif backtrack[1][i][j] == 2:
            backtrack_matrix = 2
        else:
            i -= 1
            j -= 1

    else:  # Upper backtrack matrix conditions.
        if backtrack[2][i][j] == 1:
            backtrack_matrix = 1
        j -= 1
        v_aligned = insert_indel(v_aligned, i)
        

In [53]:
for _ in range(i):
    w_aligned = insert_indel(w_aligned, 0)
for _ in range(j):
    v_aligned = insert_indel(v_aligned, 0)


In [55]:
print(max_score)
print(w_aligned)
print(v_aligned)

8
PRT---EINS
PRTWPSEIN-
